# Classification
Classify malware samples according to their labels using their reduced representations.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-05-07 10:31:13.216810: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-07 10:31:13.751840: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-05-07 10:31:14.527023: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:31:14.556571: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:31:14.556612: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

The `1-Data-Exploration.ipynb` file should have been run before this, and, as a result, the `processed-data.csv` file should have been generated.

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("../data/processed-data.csv")
raw_df

,label,hash,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,0.0,0.000000,21.743380,0.0,22.543247,2.443337,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,6.863416,0.0,11.860749,0.000000,0.000000,6.950177
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,0.0,0.000000,0.000000,0.0,5.224687,4.526221,0.000000,0.000000,...,0.000000,0.0,0.0,7.530016,13.983922,0.0,0.294426,0.000000,0.000000,16.371454
2,SALITY,e7fc7de574f44a966b198b7625bd6c595cad05bd669619...,0.0,0.000000,8.111208,0.0,20.635378,5.108455,0.000000,4.035753,...,2.569614,0.0,0.0,0.000000,3.810240,0.0,15.515412,0.000000,1.036820,7.405690
3,COBALTSTRIKE,dab956e9c864a84d12e8106a24ac3cf2950394152c62b6...,0.0,2.065497,0.858134,0.0,5.583910,0.000000,0.000000,0.000000,...,2.944685,0.0,0.0,0.221210,14.333819,0.0,9.511484,0.000000,3.637047,0.000000
4,FLYSTUDIO,5a4ef2ecbc89aecb114ab7ba9d8e1b20bd475c2e689286...,0.0,0.000000,0.012156,0.0,1.057381,0.059206,0.634038,0.742445,...,0.000000,0.0,0.0,0.000000,3.111623,0.0,2.802934,3.206999,4.481943,0.815284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,MANSABO,78514a632682d1c07ee4f782302bb6a74f2676f1a91b56...,0.0,0.000000,18.220580,0.0,0.532862,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,15.737241,18.534615,0.0,8.861106,0.000000,0.000000,0.000000
2819,TOFSEE,b7528b0cf626a97697008de2ef8281d9f2ba00dbb80171...,0.0,0.060617,4.829308,0.0,4.033799,11.053313,0.000000,0.000000,...,6.812126,0.0,0.0,0.000000,6.056214,0.0,0.000000,0.000000,2.325842,6.494234
2820,TRICKBOT,3cb4905c742c52b55c8930003224d4176220950c4cc37b...,0.0,0.000000,1.128719,0.0,4.236022,6.092369,0.000000,1.430992,...,3.938910,0.0,0.0,0.000000,3.109124,0.0,2.455679,0.000000,9.521128,6.103506
2821,TRICKBOT,46401903e85a5c457490a6934ec4dc61fdf28df83af377...,0.0,0.000000,16.154514,0.0,5.835102,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,12.208120,18.016140,0.0,4.018389,0.000000,0.000000,2.246171


For training the model, we don't need the `hash` of the sample.

In [5]:
df = raw_df.drop(columns=["hash"])
df

,label,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,dim-08,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,0.0,0.000000,21.743380,0.0,22.543247,2.443337,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,6.863416,0.0,11.860749,0.000000,0.000000,6.950177
1,DARKKOMET,0.0,0.000000,0.000000,0.0,5.224687,4.526221,0.000000,0.000000,7.718221,...,0.000000,0.0,0.0,7.530016,13.983922,0.0,0.294426,0.000000,0.000000,16.371454
2,SALITY,0.0,0.000000,8.111208,0.0,20.635378,5.108455,0.000000,4.035753,0.000000,...,2.569614,0.0,0.0,0.000000,3.810240,0.0,15.515412,0.000000,1.036820,7.405690
3,COBALTSTRIKE,0.0,2.065497,0.858134,0.0,5.583910,0.000000,0.000000,0.000000,0.000000,...,2.944685,0.0,0.0,0.221210,14.333819,0.0,9.511484,0.000000,3.637047,0.000000
4,FLYSTUDIO,0.0,0.000000,0.012156,0.0,1.057381,0.059206,0.634038,0.742445,0.116127,...,0.000000,0.0,0.0,0.000000,3.111623,0.0,2.802934,3.206999,4.481943,0.815284
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,MANSABO,0.0,0.000000,18.220580,0.0,0.532862,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,15.737241,18.534615,0.0,8.861106,0.000000,0.000000,0.000000
2819,TOFSEE,0.0,0.060617,4.829308,0.0,4.033799,11.053313,0.000000,0.000000,0.000000,...,6.812126,0.0,0.0,0.000000,6.056214,0.0,0.000000,0.000000,2.325842,6.494234
2820,TRICKBOT,0.0,0.000000,1.128719,0.0,4.236022,6.092369,0.000000,1.430992,0.000000,...,3.938910,0.0,0.0,0.000000,3.109124,0.0,2.455679,0.000000,9.521128,6.103506
2821,TRICKBOT,0.0,0.000000,16.154514,0.0,5.835102,0.000000,0.000000,0.000000,4.790173,...,0.000000,0.0,0.0,12.208120,18.016140,0.0,4.018389,0.000000,0.000000,2.246171


Separate into `X` and `y`.

In [6]:
X = df.drop(columns=["label"])
y = df["label"]

In [7]:
print(X.shape)
print(y.shape)

(2823, 32)
(2823,)


# Label Encoding and Train-Test Split

We need to encode the labels for the `y`.

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

We want to save the label encoder for use in the full model.

In [9]:
import pickle

with open("../models/classifier/label-encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

80% of the data will be saved for training, while 20% will be left for testing.

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2258, 32)
(565, 32)
(2258,)
(565,)


# Model Creation

Nothing too special here, we just use several dense layers for classification.

In [12]:
from keras import layers
from keras.models import Sequential

In [13]:
INITIAL_EPOCH = 0

def create_model():
    model = Sequential(name="Classifier")

    model.add(keras.Input((X.shape[1],), name="input"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(len(label_encoder.classes_), activation="softmax"))

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["sparse_categorical_accuracy"]
    )
    return model

In [14]:
model = create_model()
model.summary()

2024-05-07 10:31:14.863870: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:31:14.863952: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:31:14.863982: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:31:15.728455: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-07 10:31:15.728603: I external/local_xla/xla/stream_executor

Model: "Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 53)             │         3,445 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,645 (150.96 KB)

 Trainable params: 38,645 (150.96 KB)

 Non-trainable params: 0 (0.00 B)

Define callbacks.

In [15]:
import os

checkpoint_path = "../models/classifier/checkpoint.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [16]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1, min_delta=1e-4)

Load latest checkpoint if there is one.

In [17]:
# INITIAL_EPOCH = 54
# model = keras.models.load_model(checkpoint_path)

Train the model.

In [18]:
NUM_EPOCHS = 100

model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    initial_epoch=INITIAL_EPOCH,
    epochs=NUM_EPOCHS,
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/100


I0000 00:00:1715049076.566776   82399 service.cc:145] XLA service 0x7f0b2c002510 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1715049076.566814   82399 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-05-07 10:31:16.586744: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-07 10:31:17.352365: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8902


41/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 3.5535 - sparse_categorical_accuracy: 0.2268 

I0000 00:00:1715049078.550363   82399 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 3.3796 - sparse_categorical_accuracy: 0.2584
Epoch 1: val_loss improved from inf to 2.25361, saving model to ../models/classifier/checkpoint.keras
57/57 ━━━━━━━━━━━━━━━━━━━━ 5s 37ms/step - loss: 3.3704 - sparse_categorical_accuracy: 0.2602 - val_loss: 2.2536 - val_sparse_categorical_accuracy: 0.4226
Epoch 2/100
43/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.0300 - sparse_categorical_accuracy: 0.4823 
Epoch 2: val_loss improved from 2.25361 to 1.71103, saving model to ../models/classifier/checkpoint.keras
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.9863 - sparse_categorical_accuracy: 0.4965 - val_loss: 1.7110 - val_sparse_categorical_accuracy: 0.5863
Epoch 3/100
45/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.4561 - sparse_categorical_accuracy: 0.6430 
Epoch 3: val_loss improved from 1.71103 to 1.55284, saving model to ../models/classifier/checkpoint.keras
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.4515 - sparse_categorical_a

Load the best performing model.

In [19]:
model = keras.models.load_model(checkpoint_path)

In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Testing loss:      {test_loss:5.5f}")
print(f"Testing accuracy:  {test_acc:5.5f}")

18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 1.1926 - sparse_categorical_accuracy: 0.7261
Testing loss:      1.16637
Testing accuracy:  0.71858


In [21]:
model.save("../models/classifier/classifier.keras")